# HW3 Image Classification

# Training

In [1]:
_exp_name = "augment_mixup"

In [2]:
# Import necessary packages.
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [4]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.CenterCrop((384, 384)),
    transforms.ToTensor(),
    transforms.TenCrop((192, 192)),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(90),
    transforms.RandomResizedCrop((192, 192)),
    transforms.ToTensor(),
])

## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [5]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files=None, training=True):
        super(FoodDataset).__init__()
        self.training = training
        self.path = path
        if files != None:
            self.files = files
        else:
            self.files = sorted([os.path.join(path,x)
                                 for x in os.listdir(path) if x.endswith(".jpg")])
        print(f"One {path} sample",self.files[0])
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        if self.training == False:
            return torch.stack(im), label
        blender_fname = self.files[random.randint(0,  len(self.files) - 1)]
        blender_im = self.transform(Image.open(blender_fname))
        lam = 0.20
        alpha = np.random.beta(lam, lam)
        try:
            blender_label = int(blender_fname.split("/")[-1].split("_")[0])
        except:
            blender_label = -1 # test has no label
        return im * alpha + blender_im * (1 - alpha), label, blender_label, alpha

In [6]:
batch_size = 32
fold = 4
_dataset_dir = "/kaggle/input/ml2022spring-hw3b/food11"
path = os.path.join(_dataset_dir, "training")
train_files = [os.path.join(path, x)
               for x in os.listdir(path) if x.endswith(".jpg")]
path = os.path.join(_dataset_dir, "validation")
train_files += [os.path.join(path, x)
                for x in os.listdir(path) if x.endswith(".jpg")]
rng = np.random.default_rng(2333)
rng.shuffle(train_files)
segsize = len(train_files) // fold

train_loader = []
valid_loader = []
for i in range(fold):
    if i != fold - 1:
        files = train_files[i * segsize : (i + 1) * segsize]
    else:
        files = train_files[i * segsize :]
    valid_set = FoodDataset("", tfm=test_tfm, training=False, files=files)
    valid_loader.append(DataLoader(valid_set, batch_size=batch_size,
                                   shuffle=False, num_workers=0, pin_memory=True))
    train_set = FoodDataset("", tfm=train_tfm, training=True, files=files)
    train_loader.append(DataLoader(train_set, batch_size=batch_size,
                                   shuffle=True, num_workers=0, pin_memory=True))

One  sample /kaggle/input/ml2022spring-hw3b/food11/training/8_632.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/training/8_632.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/training/8_543.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/training/8_543.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/validation/2_151.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/validation/2_151.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/validation/9_449.jpg
One  sample /kaggle/input/ml2022spring-hw3b/food11/validation/9_449.jpg


In [7]:
class BCELosswithLogits(nn.Module):
    def __init__(self, reduction='mean'):
        super(BCELosswithLogits, self).__init__()
        self.reduction = reduction

    def forward(self, logits, target1, target2, alpha):
        # logits: [N, *], target: [N, *]
        target1 = target1.unsqueeze(1)
        target2 = target2.unsqueeze(1)
        alpha = alpha.unsqueeze(1)
        logits = F.log_softmax(logits, 1)
        loss1 = - logits.gather(1, target1)
        loss2 = - logits.gather(1, target2)
        loss = loss1 * alpha + loss2 * (1 - alpha)
        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        return loss
criterion = BCELosswithLogits()
valid_crit = nn.CrossEntropyLoss()

In [8]:
n_epochs = 300
learning_rate = 5e-6
weight_decay = 1e-4

models = []
optimizers = []
for i in range(fold):
    model = torchvision.models.efficientnet_b4(weights=None)
    try:
        model.load_state_dict(torch.load(f"{_exp_name}_best_{i}.ckpt"))
    except:
        print('ckpt file missing')
    model = model.to(device)
    models.append(model)
    optimizers.append(torch.optim.AdamW(model.parameters(),
                                        lr=learning_rate,
                                        weight_decay=weight_decay,
                                        eps=1e-8))

In [9]:
try:
    tmp = np.load(f"{_exp_name}_acc.npy")
    best_acc = tmp.tolist()
except:
    best_acc = [0.0 for i in range(fold)]
print(best_acc)

[0.8785199522972107, 0.8777902126312256, 0.8890367746353149, 0.8669299483299255]


In [10]:
for epoch in range(n_epochs):
    for fd in range(fold):
        train_loss = []

        for batch in tqdm(train_loader[fd]):
            imgs, labels, blender_labels, alphas = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            alphas = alphas.to(device)
            blender_labels = blender_labels.to(device)
            
            for md in range(fold):
                if md == fd:
                    continue
                model = models[md]
                optimizer = optimizers[md]
                model.train()
                logits = model(imgs).float()
                loss = criterion(logits, labels, blender_labels, alphas)

                optimizer.zero_grad()
                loss.backward()
                grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
                optimizer.step()
                train_loss.append(loss.item())

        train_loss = sum(train_loss) / len(train_loss)

        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")
    
    for md in range(fold):
        model = models[md]
        model.eval()

        valid_loss = []
        valid_accs = []

        for batch in tqdm(valid_loader[md]):
            imgs, labels = batch
            imgs = imgs.view(-1, 3, 192, 192)
            with torch.no_grad():
                logits = model(imgs.to(device)).float()
            logits = F.softmax(logits, 1)
            _, category = logits.shape
            logits = logits.view(-1, 10, category).mean(dim=1)
            loss = valid_crit(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            valid_loss.append(loss.item())
            valid_accs.append(acc)

        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
        valid_acc = valid_acc.cpu()

        print(f"[ fold {md} | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        with open(f"./{_exp_name}_log.txt","a") as log:
            if valid_acc > best_acc[md]:
                log.write(f"[ Valid {md} | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best\n")
            else:
                log.write(f"[ Valid {md} | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}\n")

        if valid_acc > best_acc[md]:
            print(f"[fold {md}]: Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"{_exp_name}_best_{md}.ckpt")
            best_acc[md] = valid_acc
            stale = 0
    np.save(f"{_exp_name}_acc.npy", np.array(best_acc))
    print(best_acc)

  0%|          | 0/104 [00:00<?, ?it/s]

[ Train | 001/300 ] loss = 0.75408


  0%|          | 0/104 [00:00<?, ?it/s]

[ Train | 001/300 ] loss = 0.78977


  0%|          | 0/104 [00:00<?, ?it/s]

...

[ fold 3 | 018/300 ] loss = 6.25969, acc = 0.87783
[tensor(0.8884), tensor(0.8883), tensor(0.8918), tensor(0.8781)]


  0%|          | 0/104 [00:00<?, ?it/s]

[ Train | 019/300 ] loss = 0.73851


  0%|          | 0/104 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm, training=False)
test_loader = DataLoader(test_set, batch_size=batch_size,
                         shuffle=False, num_workers=0, pin_memory=True)

One /kaggle/input/ml2022spring-hw3b/food11/test sample /kaggle/input/ml2022spring-hw3b/food11/test/0001.jpg


# Testing and generate prediction CSV

In [12]:
models = []
for i in range(fold):
    model = torchvision.models.efficientnet_b4(weights=None).to(device)
    model.load_state_dict(torch.load(f"{_exp_name}_best_{i}.ckpt"))
    model.eval()
    models.append(model)
prediction = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        logits = None
        imgs, _ = batch
        imgs = imgs.view(-1, 3, 192, 192)
        imgs = imgs.to(device)
        for fd in range(fold):
            with torch.no_grad():
                tlogits = models[fd](imgs)
            tlogis = F.softmax(tlogits, 1)
            _, category = tlogits.shape
            tlogits = tlogits.view(-1, 10, category).mean(dim=1)
            tlogits *= best_acc[fd]
            if logits != None:
                logits += tlogits
            else:
                logits = tlogits
        test_label = np.argmax(logits.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

  0%|          | 0/105 [00:00<?, ?it/s]

In [13]:
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

In [14]:
print(best_acc)

[tensor(0.8884), tensor(0.8883), tensor(0.8918), tensor(0.8781)]
